In [1]:
import numpy as np
import pandas as pd
from typing import Any , Literal
from src.data import load_target_file , get_target_dates

from src.data.tushare.basic.access import TradeCalendar , TradeDataAccess , ModelDataAccess , FinaDataAccess

calendar = TradeCalendar()
trade_data_access = TradeDataAccess()
model_data_access = ModelDataAccess()
fina_data_access = FinaDataAccess()
calendar.calendar_start

19901219

In [3]:
from src.data.tushare.model import TuShareCNE5_Calculator
tushare_cne5 = TuShareCNE5_Calculator()
tushare_cne5.Update('risk')


Finish risk update at date 20050104
Finish risk update at date 20050105
Finish risk update at date 20050106
Finish risk update at date 20050107
Finish risk update at date 20050110
Finish risk update at date 20050111
Finish risk update at date 20050112
Finish risk update at date 20050113
Finish risk update at date 20050114
Finish risk update at date 20050117
Finish risk update at date 20050118
Finish risk update at date 20050119
Finish risk update at date 20050120
Finish risk update at date 20050121
Finish risk update at date 20050124
Finish risk update at date 20050125
Finish risk update at date 20050126
Finish risk update at date 20050127
Finish risk update at date 20050128
Finish risk update at date 20050131
Finish risk update at date 20050201
Finish risk update at date 20050202
Finish risk update at date 20050203
Finish risk update at date 20050204
Finish risk update at date 20050216
Finish risk update at date 20050217
Finish risk update at date 20050218
Finish risk update at date 2

In [5]:
rsk = pd.read_feather('./data/DataBase/DB_models/risk_exp/2012/risk_exp.20120105.feather')
rsk.iloc[:10]

,secid,estuniv,weight,market,petro,coal,nonferrous,utility,public,steel,...,size,beta,momentum,residual_volatility,non_linear_size,book_to_price,liquidity,earnings_yield,growth,leverage
0,1,1,477.914703,1,0,0,0,0,0,0,...,0.840386,-0.472120,0.159113,-0.209820,-0.489287,-0.258295,0.289957,1.660876,0.515678,1.064224
1,2,1,700.246948,1,0,0,0,0,0,0,...,0.840386,0.070228,-0.061737,-0.308938,-0.489287,0.262514,0.211596,0.569209,0.165846,0.906542
2,4,1,6.148198,1,0,0,0,0,0,0,...,-3.698964,0.777850,0.192959,0.182589,-2.320993,-1.490264,-0.591618,-2.070229,1.005509,-1.183045
3,5,0,0.000000,1,0,0,0,0,0,0,...,-1.891241,-2.042660,0.418196,-1.018986,1.034360,-1.195547,-0.173172,-1.926430,-2.014068,-1.423251
4,6,1,38.810776,1,0,0,0,0,0,0,...,-1.770511,0.254027,-0.421254,-0.710314,1.121449,0.523630,-0.668045,1.660876,0.137248,0.495918
5,7,0,0.000000,1,0,0,0,0,0,0,...,-2.766564,-0.258178,0.367667,1.050261,-0.570277,-2.076923,-0.698384,-2.086478,0.018083,-1.693052
6,8,0,0.000000,1,0,0,0,0,0,0,...,-3.983310,0.999235,-0.753704,-0.283624,-2.320993,-1.324957,-1.073261,-2.247055,-0.179757,-1.921547
7,9,1,102.978355,1,0,0,0,0,0,0,...,-0.771988,1.893352,0.567452,1.831254,0.981290,-1.001495,1.299373,-1.602258,-0.275433,0.731750
8,10,0,0.000000,1,0,0,0,0,0,0,...,-2.403883,-2.042660,0.991700,-1.485509,0.322576,-1.876460,0.122572,-2.234237,-0.211353,-1.192546
9,11,1,7.925437,1,0,0,0,0,0,0,...,-1.975303,1.398808,-1.387071,1.212768,0.956871,-0.885509,2.556690,0.353014,1.721017,0.179217
